In [ ]:
from IPython.lib.display import join
import numpy as np
import pandas as pd
import operator
import os
from os import listdir
from os.path import isfile, join
from nltk.corpus import stopwords
from string import punctuation
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt



path = "20_newsgroups"

dirs = [f for f in listdir(path)]
dirs



info={} 
for folder in dirs:
    info[folder]=[]
    for catalog in os.listdir(os.path.join('C:\\Users\\kalya\Desktop\\project\\ml_project_2\\20_newsgroups',folder)):
        with open(os.path.join('C:\\Users\\kalya\Desktop\\project\\ml_project_2\\20_newsgroups',folder,catalog),encoding='latin-1') as catalog_open:
            info[folder].append(catalog_open.read())
print(len(info[dirs[1]]))



import nltk
nltk.download('stopwords')
punctuations=list(punctuation)
stopWords=stopwords.words('english')
stopWords=stopWords+punctuations 



stopWords=stopWords+['a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', "aren't", 'as', 'at',
 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', "can't", 'cannot', 'could', "couldn't", 'did', "didn't", 'do', 'does', "doesn't", 'doing', "don't", 'down', 'during',
 'each', 'few', 'for', 'from', 'further','had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having', 'he', "he'd", "he'll", "he's", 'her', 'here', "here's",
 'hers', 'herself', 'him', 'himself', 'his', 'how', "how's",'i', "i'd", "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', "isn't", 'it', "it's", 'its', 'itself',
 "let's", 'me', 'more', 'most', "mustn't", 'my', 'myself','no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'ought', 'our', 'ours' 'ourselves', 'out', 'over', 'own',
 'same', "shan't", 'she', "she'd", "she'll", "she's", 'should', "shouldn't", 'so', 'some', 'such', 'than', 'that',"that's", 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', "there's", 'these', 'they', "they'd", 
 "they'll", "they're", "they've", 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 'very', 
 'was', "wasn't", 'we', "we'd", "we'll", "we're", "we've", 'were', "weren't", 'what', "what's", 'when', "when's", 'where',
 "where's", 'which', 'while', 'who', "who's", 'whom', 'why', "why's",'will', 'with', "won't", 'would', "wouldn't", 
 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'hundred', 'thousand', '1st', '2nd', '3rd',
 '4th', '5th', '6th', '7th', '8th', '9th', '10th']


lexicon={}
for i in range(len(info)):
    for item in info[dirs[i]]: 
        for piece in item.split():
            if piece.lower() not in stopWords and len(piece.lower()) >= 5:
                if piece.lower() not in lexicon:
                    lexicon[piece.lower()]=1
                else:
                    lexicon[piece.lower()]=lexicon[piece.lower()]+1
len(lexicon)



lex_sort=sorted(lexicon.items(),key=operator.itemgetter(1),reverse=True)
feature_list=[]
for key in lex_sort:
    feature_list=feature_list+[key[0]]
feature_list=feature_list[0:2000] 



Y=[]  
for i in range(len(info)):
    for item in info[dirs[i]]:
        Y=Y+[dirs[i]]
Y=np.array(Y)



feature_table = pd.DataFrame(columns = feature_list)
for folder in dirs: 
    for catalog in os.listdir(os.path.join('C:\\Users\\kalya\Desktop\\project\\ml_project_2\\20_newsgroups',folder)):
        feature_table.loc[len(feature_table)] = np.zeros(len(feature_list))
        with open(os.path.join('C:\\Users\\kalya\Desktop\\project\\ml_project_2\\20_newsgroups',folder,catalog),encoding='latin-1') as catalog_open:
            for piece in catalog_open.read().split():
                if piece.lower() in feature_list:
                    feature_table[piece.lower()][len(feature_table)-1]=feature_table[piece.lower()][len(feature_table)-1]+ 1 
feature_table




feat=feature_table.values
feat
x_train,x_test,y_train,y_test=train_test_split(feat,Y,random_state=0,test_size=0.80)




def nb_fit(x_train,y_train):
    result={}
    result["total_data"]=len(y_train)
    class_labels=set(y_train)
    for present_names in class_labels:
        result[present_names]={}
        present_records=(y_train==present_names)
        x_train_current=x_train[present_records]
        y_train_current=y_train[present_records]
        complete_word_set=0
        for i in range(len(feature_list)):
            result[present_names][feature_list[i]]=x_train_current[:,i].sum()
            complete_word_set=complete_word_set+x_train_current[:,i].sum()
        result[present_names]["count_sum"]=complete_word_set
    return result



def cal_prob(x,mapItemsVals,present_cat):
    sol=np.log(mapItemsVals[present_cat]["count_sum"])-np.log(mapItemsVals["total_data"])
    for i in range(len(feature_list)):
        current_word_count=mapItemsVals[present_cat][feature_list[i]]+1
        complete_piece_num=mapItemsVals[present_cat]["count_sum"]+len(feature_list)
        present_item_prob=np.log(current_word_count)-np.log(complete_piece_num)
        for j in range(int(x[i])): 
            sol=sol+present_item_prob
    return sol




def oneClassPred(x,mapItemsVals):
    best_class=-500
    probability_optimal=-500
    primarySprint=True
    feasible_cat=mapItemsVals.keys()
    for present_cat in feasible_cat:
        if present_cat=="total_data":
            continue
        current_class_prob=cal_prob(x,mapItemsVals,present_cat)
        if(primarySprint==True or current_class_prob>probability_optimal):
            best_class=present_cat
            probability_optimal=current_class_prob
        primarySprint=False
    return best_class



def prediction(X_test,mapItemsVals):
    Y_pred=[]
    num = 0
    for x in X_test:
        Y_pred=Y_pred+[oneClassPred(x,mapItemsVals)]
    return Y_pred





mapItemsVals=nb_fit(x_train,y_train)
y_pred=prediction(x_test,mapItemsVals)


print(confusion_matrix(y_pred,y_test))
print(classification_report(y_pred,y_test))







